Load dataframes with acf and calculate tau

In [1]:
import pandas as pd
import numpy as np

# import from scripts
import os
current_wd = os.getcwd()
os.chdir(os.path.abspath("..\\..\\..\\isttc\\scripts"))
from calculate_tau import fit_single_exp, func_single_exp_monkey
from cfg_global import project_folder_path
os.chdir(current_wd)

In [2]:
results_folder = project_folder_path + 'results\\monkey\\fixation_period_1000ms\\'

### Functions

In [ ]:
def calc_tau_per_unit(acf_df_, acf_cols_, start_idx_):
    acf_2d = acf_df_[acf_cols_].values
    acf_2d = acf_2d.astype(np.float64) # RuntimeWarning: overflow encountered in matmul sttc start_idx 2
    n_units = acf_2d.shape[0]
    print('Calculating taus for {}'.format(acf_2d.shape))
    
    fit_popt_a_l, fit_popt_b_l, fit_popt_c_l = [],[],[]
    fit_tau_l = []
    fit_r_squared_l = []
    fit_log_message_l = []
    
    for i in range(n_units):
        fit_popt, fit_pcov, tau, fit_r_squared, log_message = fit_single_exp(acf_2d[i,:], start_idx_, func_single_exp_monkey)
        if  type(fit_popt) == np.ndarray:
            fit_popt_a_l.append(fit_popt[0])
            fit_popt_b_l.append(fit_popt[1])
            fit_popt_c_l.append(fit_popt[2])
        else:
            fit_popt_a_l.append(np.nan)
            fit_popt_b_l.append(np.nan)
            fit_popt_c_l.append(np.nan)
        fit_tau_l.append(tau)
        fit_r_squared_l.append(fit_r_squared)
        fit_log_message_l.append(log_message)
    
    data_df = np.vstack((fit_popt_a_l, fit_popt_b_l, fit_popt_c_l, fit_tau_l, fit_r_squared_l, fit_log_message_l)).T
    tau_df = pd.DataFrame(data_df, columns=['fit_a', 'fit_b', 'fit_c','tau', 'r_squared', 'log_message'])
    tau_df.insert(0, 'unit_id', acf_df_['unit_id'].values)

    for col in ['fit_a', 'fit_b', 'fit_c','tau', 'r_squared']:
        tau_df[col] = tau_df[col].astype(float) 
    
    return tau_df

In [ ]:
# different way of calculating tau per area 
def calc_tau_area(acf_df_, acf_cols_, start_idx_=1):
    acf_2d = acf_df_[acf_cols_].values
    print(acf_2d.shape)
    
    t = np.linspace(start_idx_, acf_2d.shape[1]-start_idx_, acf_2d.shape[1]-start_idx_).astype(int)
    # print(t)
    
    # make 1d for curve_fit
    acf_1d = np.hstack(acf_2d[:,1:])
    print(acf_1d.shape)
    t_1d = np.tile(t, reps=acf_2d.shape[0])
    print(t_1d.shape)
    
    popt, pcov = curve_fit(func_single_exp, t_1d, acf_1d, maxfev=1000000000) # I used 5000, now it is like in Siegle
    tau = 1 / popt[1]
    
    # fit r-squared
    y_pred = func_single_exp(t_1d, *popt)
    fit_r_squared = r2_score(acf_1d, y_pred)

    return tau, popt, fit_r_squared

### Calculate tau

#### One tau per area

2 methods of doing that:
* average ACF over units and fit
* fit using all ACFs from all neurons

In [ ]:
n_lags = 20
bin_size = 50
acf_cols = ['acf_' + str(i) for i in range(n_lags)]
print('acf_cols {}'.format(acf_cols))

##### average ACF over units and fit

In [ ]:
area = 'pfdl' 

pearsonr_trial_avg_df = pd.read_pickle(results_folder + 'binned\\' + area + '\\acf\\acf_pearsonr_trial_avg_1000ms_with_empty_50ms_20lags_df.pkl')
pearsonr_trial_avg_2d_mean = np.nanmean(pearsonr_trial_avg_df[acf_cols].values, axis=0) # todo remove all rows with NaNs? 
pfdl_pearsonr_trial_avg_fit_popt, _, pfdl_pearsonr_trial_avg_tau, pfdl_pearsonr_trial_avg_fit_r_squared, _ = fit_single_exp(pearsonr_trial_avg_2d_mean, 
                                                                            start_idx_=2, exp_fun_=func_single_exp_monkey)
pfdl_pearsonr_trial_avg_tau_ms = pfdl_pearsonr_trial_avg_tau*bin_size
print('tau {}, popt {}, fit_r_squared {}'.format(pfdl_pearsonr_trial_avg_tau_ms, 
                                                             pfdl_pearsonr_trial_avg_fit_popt, 
                                                             pfdl_pearsonr_trial_avg_fit_r_squared))

sttc_trial_avg_df = pd.read_pickle(results_folder + 'non_binned\\' + area + '\\acf\\acf_sttc_trial_avg_1000ms_with_empty_50ms_20lags_df.pkl')
sttc_trial_avg_2d_mean = np.nanmean(sttc_trial_avg_df[acf_cols].values, axis=0) # todo remove all rows with NaNs? 
pfdl_sttc_trial_avg_fit_popt, _, pfdl_sttc_trial_avg_tau, pfdl_sttc_trial_avg_fit_r_squared, _ = fit_single_exp(sttc_trial_avg_2d_mean, 
                                                                                                                start_idx_=2, 
                                                                                                                exp_fun_=func_single_exp_monkey)
pfdl_sttc_trial_avg_tau_ms = pfdl_sttc_trial_avg_tau*bin_size
print('tau {}, popt {}, fit_r_squared {}'.format(pfdl_sttc_trial_avg_tau_ms, 
                                                             pfdl_sttc_trial_avg_fit_popt, 
                                                             pfdl_sttc_trial_avg_fit_r_squared))

sttc_trial_concat_df = pd.read_pickle(results_folder + 'non_binned\\' + area + '\\acf\\acf_sttc_trial_concat_1000ms_with_empty_50ms_20lags_df.pkl')
sttc_trial_concat_2d_mean = np.nanmean(sttc_trial_concat_df[acf_cols].values, axis=0) # todo remove all rows with NaNs? 
pfdl_sttc_trial_concat_fit_popt, _, pfdl_sttc_trial_concat_tau, pfdl_sttc_trial_concat_fit_r_squared, _ = fit_single_exp(sttc_trial_concat_2d_mean, 
                                                                                                                start_idx_=2, 
                                                                                                                exp_fun_=func_single_exp_monkey)
pfdl_sttc_trial_concat_tau_ms = pfdl_sttc_trial_concat_tau*bin_size
print('tau {}, popt {}, fit_r_squared {}'.format(pfdl_sttc_trial_concat_tau_ms, 
                                                             pfdl_sttc_trial_concat_fit_popt, 
                                                             pfdl_sttc_trial_concat_fit_r_squared))

In [ ]:
area = 'pfp' 

pearsonr_trial_avg_df = pd.read_pickle(results_folder + 'binned\\' + area + '\\acf\\acf_pearsonr_trial_avg_1000ms_with_empty_50ms_20lags_df.pkl')
pearsonr_trial_avg_2d_mean = np.nanmean(pearsonr_trial_avg_df[acf_cols].values, axis=0) # todo remove all rows with NaNs? 
pfp_pearsonr_trial_avg_fit_popt, _, pfp_pearsonr_trial_avg_tau, pfp_pearsonr_trial_avg_fit_r_squared, _ = fit_single_exp(pearsonr_trial_avg_2d_mean, 
                                                                            start_idx_=2, exp_fun_=func_single_exp_monkey)
pfp_pearsonr_trial_avg_tau_ms = pfp_pearsonr_trial_avg_tau*bin_size
print('tau {}, popt {}, fit_r_squared {}'.format(pfp_pearsonr_trial_avg_tau_ms, 
                                                             pfp_pearsonr_trial_avg_fit_popt, 
                                                             pfp_pearsonr_trial_avg_fit_r_squared))

sttc_trial_avg_df = pd.read_pickle(results_folder + 'non_binned\\' + area + '\\acf\\acf_sttc_trial_avg_1000ms_with_empty_50ms_20lags_df.pkl')
sttc_trial_avg_2d_mean = np.nanmean(sttc_trial_avg_df[acf_cols].values, axis=0) # todo remove all rows with NaNs? 
pfp_sttc_trial_avg_fit_popt, _, pfp_sttc_trial_avg_tau, pfp_sttc_trial_avg_fit_r_squared, _ = fit_single_exp(sttc_trial_avg_2d_mean, 
                                                                                                                start_idx_=2, 
                                                                                                                exp_fun_=func_single_exp_monkey)
pfp_sttc_trial_avg_tau_ms = pfp_sttc_trial_avg_tau*bin_size
print('tau {}, popt {}, fit_r_squared {}'.format(pfp_sttc_trial_avg_tau_ms, 
                                                             pfp_sttc_trial_avg_fit_popt, 
                                                             pfp_sttc_trial_avg_fit_r_squared))

sttc_trial_concat_df = pd.read_pickle(results_folder + 'non_binned\\' + area + '\\acf\\acf_sttc_trial_concat_1000ms_with_empty_50ms_20lags_df.pkl')
sttc_trial_concat_2d_mean = np.nanmean(sttc_trial_concat_df[acf_cols].values, axis=0) # todo remove all rows with NaNs? 
pfp_sttc_trial_concat_fit_popt, _, pfp_sttc_trial_concat_tau, pfp_sttc_trial_concat_fit_r_squared, _ = fit_single_exp(sttc_trial_concat_2d_mean, 
                                                                                                                start_idx_=2, 
                                                                                                                exp_fun_=func_single_exp_monkey)
pfp_sttc_trial_concat_tau_ms = pfp_sttc_trial_concat_tau*bin_size
print('tau {}, popt {}, fit_r_squared {}'.format(pfp_sttc_trial_concat_tau_ms, 
                                                             pfp_sttc_trial_concat_fit_popt, 
                                                             pfp_sttc_trial_concat_fit_r_squared))

#### One tau per unit

3 methods of doing that:
* Pearsonr trial average (as in the paper)
* STTC trial average
* STTC trial concat

todo: take ACF average over trails for a unit and fit? Or fit all ACFs with one function? (this requires calculating ACF per trial)

In [ ]:
n_lags = 20
bin_size = 50
acf_cols = ['acf_' + str(i) for i in range(n_lags)]
print('acf_cols {}'.format(acf_cols))

In [ ]:
area = 'pfdl' 

# pearsonr trial avg
pearsonr_trial_avg_df = pd.read_pickle(results_folder + 'binned\\' + area + '\\acf\\acf_pearsonr_trial_avg_1000ms_with_empty_50ms_20lags_df.pkl')
tau_pearsonr_trial_avg_df = calc_tau_per_unit(pearsonr_trial_avg_df, acf_cols, start_idx_=1)
tau_pearsonr_trial_avg_df['tau_ms'] = tau_pearsonr_trial_avg_df['tau'] * bin_size
tau_pearsonr_trial_avg_df.to_pickle(results_folder + 'binned\\' + area + '\\taus\\tau_pearsonr_trial_avg_1000ms_with_empty_50ms_20lags_df.pkl')

# sttc trial avg
sttc_trial_avg_df = pd.read_pickle(results_folder + 'non_binned\\' + area + '\\acf\\acf_sttc_trial_avg_1000ms_with_empty_50ms_20lags_df.pkl')
tau_sttc_trial_avg_df = calc_tau_per_unit(sttc_trial_avg_df, acf_cols, start_idx_=1)
tau_sttc_trial_avg_df['tau_ms'] = tau_sttc_trial_avg_df['tau'] * bin_size
tau_sttc_trial_avg_df.to_pickle(results_folder + 'non_binned\\' + area + '\\taus\\tau_sttc_trial_avg_1000ms_with_empty_50ms_20lags_df.pkl')

# sttc trial concat
sttc_trial_concat_df = pd.read_pickle(results_folder + 'non_binned\\' + area + '\\acf\\acf_sttc_trial_concat_1000ms_with_empty_50ms_20lags_df.pkl')
tau_sttc_trial_concat_df = calc_tau_per_unit(sttc_trial_concat_df, acf_cols, start_idx_=1)
tau_sttc_trial_concat_df['tau_ms'] = tau_sttc_trial_concat_df['tau'] * bin_size
tau_sttc_trial_concat_df.to_pickle(results_folder + 'non_binned\\' + area + '\\taus\\tau_sttc_trial_concat_1000ms_with_empty_50ms_20lags_df.pkl')

In [ ]:
area = 'pfp' 

# pearsonr trial avg
pearsonr_trial_avg_df = pd.read_pickle(results_folder + 'binned\\' + area + '\\acf\\acf_pearsonr_trial_avg_1000ms_with_empty_50ms_20lags_df.pkl')
tau_pearsonr_trial_avg_df = calc_tau_per_unit(pearsonr_trial_avg_df, acf_cols, start_idx_=1)
tau_pearsonr_trial_avg_df['tau_ms'] = tau_pearsonr_trial_avg_df['tau'] * bin_size
tau_pearsonr_trial_avg_df.to_pickle(results_folder + 'binned\\' + area + '\\taus\\tau_pearsonr_trial_avg_1000ms_with_empty_50ms_20lags_df.pkl')

# sttc trial avg
sttc_trial_avg_df = pd.read_pickle(results_folder + 'non_binned\\' + area + '\\acf\\acf_sttc_trial_avg_1000ms_with_empty_50ms_20lags_df.pkl')
tau_sttc_trial_avg_df = calc_tau_per_unit(sttc_trial_avg_df, acf_cols, start_idx_=1)
tau_sttc_trial_avg_df['tau_ms'] = tau_sttc_trial_avg_df['tau'] * bin_size
tau_sttc_trial_avg_df.to_pickle(results_folder + 'non_binned\\' + area + '\\taus\\tau_sttc_trial_avg_1000ms_with_empty_50ms_20lags_df.pkl')

# sttc trial concat
sttc_trial_concat_df = pd.read_pickle(results_folder + 'non_binned\\' + area + '\\acf\\acf_sttc_trial_concat_1000ms_with_empty_50ms_20lags_df.pkl')
tau_sttc_trial_concat_df = calc_tau_per_unit(sttc_trial_concat_df, acf_cols, start_idx_=1)
tau_sttc_trial_concat_df['tau_ms'] = tau_sttc_trial_concat_df['tau'] * bin_size
tau_sttc_trial_concat_df.to_pickle(results_folder + 'non_binned\\' + area + '\\taus\\tau_sttc_trial_concat_1000ms_with_empty_50ms_20lags_df.pkl')

#### One tau per trial

3 methods of doing that:
* Pearsonr
* ACF formula
* iSTTC

In [ ]:
n_lags = 20
bin_size = 50
acf_cols = ['acf_' + str(i) for i in range(n_lags)]
print('acf_cols {}'.format(acf_cols))

In [ ]:
area = 'pfdl' 

# pearsonr 
# pearsonr_per_trial_df = pd.read_pickle(results_folder + 'binned\\' + area + '\\acf\\acf_pearsonr_per_trial_1000ms_with_empty_50ms_20lags_df.pkl')
# print(f'N rows with NaN {pearsonr_per_trial_df.isnull().any(axis=1).sum()} out of {len(pearsonr_per_trial_df)}, dropping before tau calc ...')
# pearsonr_per_trial_df.dropna(inplace=True)
# pearsonr_per_trial_df.reset_index(drop=True, inplace=True)
# tau_pearsonr_per_trial_df = calc_tau_per_unit(pearsonr_per_trial_df, acf_cols[:-1], start_idx_=1)
# tau_pearsonr_per_trial_df['tau_ms'] = tau_pearsonr_per_trial_df['tau'] * bin_size
# tau_pearsonr_per_trial_df.insert(1, 'trial_id', pearsonr_per_trial_df['trial_id'])
# tau_pearsonr_per_trial_df.insert(2, 'condition_id', pearsonr_per_trial_df['condition_id'])
# tau_pearsonr_per_trial_df.insert(3, 'spike_count', pearsonr_per_trial_df['spike_count'])
# tau_pearsonr_per_trial_df.insert(4, 'fr_hz', pearsonr_per_trial_df['fr_hz'])
# print(f'N rows with NaN {tau_pearsonr_per_trial_df.isnull().any(axis=1).sum()} out of {len(tau_pearsonr_per_trial_df)}')
# tau_pearsonr_per_trial_df.to_pickle(results_folder + 'binned\\' + area + '\\taus\\tau_pearsonr_per_trial_1000ms_with_empty_50ms_20lags_df.pkl')

# # acf formula
# pcf_proper_per_trial_df = pd.read_pickle(results_folder + 'binned\\' + area + '\\acf\\acf_proper_per_trial_1000ms_with_empty_50ms_20lags_df.pkl')
# print(f'N rows with NaN {pcf_proper_per_trial_df.isnull().any(axis=1).sum()} out of {len(pcf_proper_per_trial_df)}, dropping before tau calc ...')
# pcf_proper_per_trial_df.dropna(inplace=True)
# pcf_proper_per_trial_df.reset_index(drop=True, inplace=True)
# tau_acf_proper_per_trial_df = calc_tau_per_unit(pcf_proper_per_trial_df, acf_cols, start_idx_=1)
# tau_acf_proper_per_trial_df['tau_ms'] = tau_acf_proper_per_trial_df['tau'] * bin_size
# tau_acf_proper_per_trial_df.insert(1, 'trial_id', pcf_proper_per_trial_df['trial_id'])
# tau_acf_proper_per_trial_df.insert(2, 'condition_id', pcf_proper_per_trial_df['condition_id'])
# tau_acf_proper_per_trial_df.insert(3, 'spike_count', pcf_proper_per_trial_df['spike_count'])
# tau_acf_proper_per_trial_df.insert(4, 'fr_hz', pcf_proper_per_trial_df['fr_hz'])
# print(f'N rows with NaN {tau_acf_proper_per_trial_df.isnull().any(axis=1).sum()} out of {len(tau_acf_proper_per_trial_df)}')
# tau_acf_proper_per_trial_df.to_pickle(results_folder + 'binned\\' + area + '\\taus\\tau_acf_proper_per_trial_1000ms_with_empty_50ms_20lags_df.pkl')

# isttc
sttc_per_trial_df = pd.read_pickle(results_folder + 'non_binned\\' + area + '\\acf\\acf_isttc_per_trial_1000ms_with_empty_50ms_20lags_df.pkl')
print(f'N rows with NaN {sttc_per_trial_df.isnull().any(axis=1).sum()} out of {len(sttc_per_trial_df)}, dropping before tau calc ...')
sttc_per_trial_df.dropna(inplace=True)
sttc_per_trial_df.reset_index(drop=True, inplace=True)
tau_isttc_per_trial_df = calc_tau_per_unit(sttc_per_trial_df, acf_cols, start_idx_=1)
tau_isttc_per_trial_df['tau_ms'] = tau_isttc_per_trial_df['tau'] * bin_size
tau_isttc_per_trial_df.insert(1, 'trial_id', sttc_per_trial_df['trial_id'])
tau_isttc_per_trial_df.insert(2, 'condition_id', sttc_per_trial_df['condition_id'])
tau_isttc_per_trial_df.insert(3, 'spike_count', sttc_per_trial_df['spike_count'])
tau_isttc_per_trial_df.insert(4, 'fr_hz', sttc_per_trial_df['fr_hz'])
print(f'N rows with NaN {tau_isttc_per_trial_df.isnull().any(axis=1).sum()} out of {len(tau_isttc_per_trial_df)}')
tau_isttc_per_trial_df.to_pickle(results_folder + 'non_binned\\' + area + '\\taus\\tau_isttc_per_trial_1000ms_with_empty_50ms_20lags_df.pkl')

In [ ]:
tau_acf_proper_per_trial_df

In [ ]:
pearsonr_per_trial_df

### todo (metrics)

In [ ]:
n_rows_with_nans = pearsonr_trial_avg_df[acf_cols].isnull().any(axis=1).sum()
n_rows_with_nans_perc = n_rows_with_nans / len(pearsonr_trial_avg_df) * 100
# acf_df.dropna(inplace=True)

n_rows_with_nans_tau = tau_pearsonr_trial_avg_df['tau_ms'].isnull().sum()
n_rows_with_nans_tau_perc = n_rows_with_nans_tau / len(tau_pearsonr_trial_avg_df) * 100
# tau_df.dropna(inplace=True)

print('acf n_rows_with_nans_perc {}, tau n_rows_with_nans_tau_perc {}'.format(n_rows_with_nans_perc, n_rows_with_nans_tau_perc))

In [ ]:
n_rows_with_nans = sttc_trial_avg_df[acf_cols].isnull().any(axis=1).sum()
n_rows_with_nans_perc = n_rows_with_nans / len(sttc_trial_avg_df) * 100
# acf_df.dropna(inplace=True)

n_rows_with_nans_tau = tau_sttc_trial_avg_df['tau_ms'].isnull().sum()
n_rows_with_nans_tau_perc = n_rows_with_nans_tau / len(tau_sttc_trial_avg_df) * 100
# tau_df.dropna(inplace=True)

print('acf n_rows_with_nans_perc {}, tau n_rows_with_nans_tau_perc {}'.format(n_rows_with_nans_perc, n_rows_with_nans_tau_perc))